In [1]:
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import math

In [2]:
json_request = {
    'allocation_weights' : [0.3,0.3,0.4],
    'codelist' : ['ABAQ','ABQI','ABQX'],
    'benchmark': 'None',
    'initial_amount' : 10000,
    'start_date' : '2019-03-13',
    'end_date' : '2020-03-13',
    'rebalance' : True,
    'rebalance_frequency' : 'Monthly'
}

In [3]:
# Backtesting Portfolio

# Define portfolio, start_date, end_date

allocation_weights = json_request['allocation_weights']
initial_amount = json_request['initial_amount']
start_date = json_request['start_date']
end_date = json_request['end_date']
codelist = json_request['codelist']
benchmark = json_request['benchmark']

rebalance = json_request['rebalance']
rebalance_frequency = json_request['rebalance_frequency']

with open('data/key.txt', 'r') as file:
    api_key = file.read()

In [4]:
indexdata = pd.DataFrame()

quandl_code = codelist
if benchmark != 'None':
    quandl_code = codelist + [benchmark]

for x in range(0,len(quandl_code)):
    quandl_request = (
    'https://www.quandl.com/api/v3/datasets/NASDAQOMX/'
    f'{quandl_code[x]}?start_date={start_date}&end_date={end_date}&api_key={api_key}'
    )
    
    response = requests.get(quandl_request).json()
    
    response_df = pd.DataFrame(response['dataset']['data'])
    response_df = response_df[[0,1]]
    if x == len(quandl_code)-1:
        response_df.columns = ['date','benchmark']
    else:
        response_df.columns = ['date',quandl_code[x]]
    
    if x == 0:
        indexdata = indexdata.append(response_df)
    else:
        indexdata = pd.merge(indexdata,response_df)

In [5]:
month_frequency = {
    'Yearly': 12,
    'Quarterly': 3,
    'Monthly': 1
}

In [6]:
end_date = datetime.strptime(end_date,'%Y-%m-%d')
start_date = datetime.strptime(start_date,'%Y-%m-%d')
num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
if (end_date.day < start_date.day):
    num_months = num_months - 1

In [7]:
event_count = int(math.floor(num_months / month_frequency[rebalance_frequency]))

rebalance_list =[start_date.strftime("%Y-%m-%d")]
rebalance_date = start_date
for x in range(0,event_count):
    rebalance_date = rebalance_date + relativedelta(months=+month_frequency[rebalance_frequency])
    rebalance_list.append(rebalance_date.strftime("%Y-%m-%d"))

In [8]:
asset_projection = []

asset_value = [x * initial_amount for x in allocation_weights]

In [9]:
indexdata

,date,ABAQ,ABQI,benchmark
0,2020-03-13,245.78,1704.43,2137.43
1,2020-03-12,224.39,1554.21,1948.21
2,2020-03-11,245.22,1694.39,2122.16
3,2020-03-10,261.19,1807.17,2263.39
4,2020-03-09,250.82,1731.57,2168.70
...,...,...,...,...
249,2019-03-19,332.06,2315.03,2828.16
250,2019-03-18,340.11,2374.76,2901.14
251,2019-03-15,336.45,2346.99,2867.17
252,2019-03-14,336.27,2346.24,2866.13


In [10]:
for x in range(0,len(rebalance_list)-1):
    asset_value = [x * initial_amount for x in allocation_weights]
    
    index_subset = indexdata[indexdata['date'] >= rebalance_list[0+x]][indexdata['date'] < rebalance_list[1+x]]
    index_subset = index_subset.sort_values(by='date').reset_index(drop=True)

    allocation = pd.DataFrame()

    for i in range(0,len(allocation_weights)):
        indexcode = index_subset.columns[i+1]
        scaling_factor = asset_value[i] / index_subset[indexcode][0]
        allocation[f'allocation {i}'] = index_subset[indexcode] * scaling_factor
    
    asset_projection.append(allocation)
    
    asset_value = allocation.iloc[len(allocation)-1].sum()

In [11]:
asset_projection = pd.concat(asset_projection).reset_index(drop=True)

In [12]:
asset_projection['output'] = asset_projection.sum(axis = 1)
asset_projection['date'] = indexdata['date']

In [13]:
if benchmark != 'None':
    scaling_factor = initial_amount / indexdata.sort_values(by='date').reset_index(drop=True)['benchmark'][0]
    asset_projection['benchmark'] = indexdata.sort_values(by='date').reset_index(drop=True)['benchmark'] * scaling_factor

In [15]:
output_field = ['date','output']

if benchmark != 'None':
    output_field = output_field +['benchmark']

asset_projection[['date','output']].to_json(orient='index')

'{"0":{"date":"2020-03-13","output":10000.0},"1":{"date":"2020-03-12","output":10008.2205596732},"2":{"date":"2020-03-11","output":10012.2401275861},"3":{"date":"2020-03-10","output":10127.9044578072},"4":{"date":"2020-03-09","output":9877.6563712957},"5":{"date":"2020-03-06","output":9610.7443438134},"6":{"date":"2020-03-05","output":9533.3195633008},"7":{"date":"2020-03-04","output":9122.2873250952},"8":{"date":"2020-03-03","output":9174.2205020346},"9":{"date":"2020-03-02","output":9420.5941105213},"10":{"date":"2020-02-28","output":9442.5764114239},"11":{"date":"2020-02-27","output":9555.5302640863},"12":{"date":"2020-02-26","output":9519.517084959},"13":{"date":"2020-02-25","output":9744.1714436799},"14":{"date":"2020-02-24","output":9708.6112972842},"15":{"date":"2020-02-21","output":9730.9963732098},"16":{"date":"2020-02-20","output":9860.0945262139},"17":{"date":"2020-02-19","output":9922.4133346091},"18":{"date":"2020-02-18","output":9917.0968810837},"19":{"date":"2020-02-14",

In [16]:
asset_projection

,allocation 0,allocation 1,allocation 2,output,date
0,3000.000000,3000.000000,4000.000000,10000.000000,2020-03-13
1,3002.232010,3001.573554,4004.414996,10008.220560,2020-03-12
2,3003.839057,3002.533038,4005.868032,10012.240128,2020-03-11
3,3036.515684,3038.059539,4053.329235,10127.904458,2020-03-10
4,2964.644962,2961.646219,3951.365191,9877.656371,2020-03-09
...,...,...,...,...,...
248,2474.443776,2464.992415,3291.503130,8230.939321,2019-03-20
249,2154.625891,2141.477310,2861.138413,7157.241614,2019-03-19
250,2243.707585,2234.973781,2986.061729,7464.743096,2019-03-18
251,2106.520058,2095.495845,2799.738781,7001.754684,2019-03-15
